In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
le=LabelEncoder()
scaler=StandardScaler()
norm=MinMaxScaler()
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(sampling_strategy='minority')
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(sampling_strategy='majority')
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB
gnb=GaussianNB()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
def ML_Monte_Carlo(X,y):
    object_columns=X.dtypes[X.dtypes==np.object].index
    integer_columns=X.dtypes[(X.dtypes==np.int64) | (X.dtypes==np.float64)].index
    #Standardization, Normalization and RAW Data
    raw=input("Do you want to transform the data, if YES then please specify the techique---")
    if raw=="YES" or raw=="yes" or raw=="y" or raw=="Yes" or raw=="Y":
        x1=input("Which Transfromation Technique you want to use, Stan/Norm::::")
        if x1=="Stan" or x1=="stan" or x1=="STAN":
            for i in integer_columns:
                X[i]=scaler.fit_transform(X[[i]])
        else:
            for i in integer_columns:
                X[i]=norm.fit_transform(X[[i]])
    else:
        pass
    #Label Encoder and Dummy Variable
    xx=input("Do you want to proceed with Label Encoder? Y or N::::")
    if xx=="Y" or xx=="y" or xx=="YES" or xx=="yes" or xx=="Yes":
        for i in object_columns:
            X[i]=le.fit_transform(X[i])
    else:
        for ii in object_columns:
            x1=pd.get_dummies(X[ii],drop_first=True)
            X=pd.concat([X,x1],axis=1)
        X.drop(columns=object_columns,inplace=True)
    #Treating Over Samples and Under Samples
    first=y.value_counts().index[0]
    first_iter=y.value_counts().values[0]
    second=y.value_counts().index[1]
    second_iter=y.value_counts().values[1]
    print('The iteration of {} category is {} that means the concentration of {} category on overall data is {}%'.format(first,first_iter,first,np.round((first_iter/y.shape[0])*100,3)))
    print("The iteration of {} category is {} that means the concentration of {} category on overall data is {}%".format(second,second_iter,second,np.round(second_iter/y.shape[0],3)*100))
    if np.round(first_iter/y.shape[0],3)>=0.65:
        imbalance=input("Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! ")
        if imbalance=="Yes" or imbalance=="YES" or imbalance=="Y" or imbalance=="y" or imbalance=="yes":
            X_train_reshape, X_test, y_train_reshape, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)
        else:
            answer=input("Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ")
            if answer=="ROS" or answer=="ros" or answer=="Ros":
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                X_train_reshape,y_train_reshape=ros.fit_resample(X_train,y_train)
            else:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                X_train_reshape,y_train_reshape=rus.fit_resample(X_train,y_train)
    else:
        pass
    #Logistic Regression
    log.fit(X_train_reshape,y_train_reshape)
    log_pred=log.predict(X_test)
    log_accuracy=accuracy_score(y_test,log_pred)
    print()
    print("Accuracy of Logistic Regression is ",np.round(log_accuracy,3))
    
    #RandomForestClassification
    rfc.fit(X_train_reshape,y_train_reshape)
    rfc_pred=rfc.predict(X_test)
    rfc_accuracy=accuracy_score(y_test,rfc_pred)
    print("Accuracy of RandomForestClassification is ",np.round(rfc_accuracy,3))
    
    #NaiveBayes Classification
    gnb.fit(X_train_reshape,y_train_reshape)
    gnb_pred=gnb.predict(X_test)
    gnb_accuracy=accuracy_score(y_test,gnb_pred)
    print("Accuracy of NaiveBayesClassification is ",np.round(gnb_accuracy,3))

    #KNN Classifier
    knn_num=int(input("For how many time you want to iterate KNN Algorithm? "))
    Accu = []
    for i in range(1,knn_num):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train_reshape,y_train_reshape)
        pred_i = knn.predict(X_test)
        Accu.append(accuracy_score(y_test,pred_i))
    print("Highest accuracy from K-Nearest Neighbor is {} ".format(sorted(Accu,reverse=True)[0]))

In [5]:
def ML_Cross_Val(X,y):
    object_columns=X.dtypes[X.dtypes==np.object].index
    integer_columns=X.dtypes[(X.dtypes==np.int64) | (X.dtypes==np.float64)].index    
    
    #Standardization and Normalization
    repeat=int(input("How many batches of data you want in Monte Carlo simulation?  "))
    raw=input("Do you want to transform the data, if YES then please specify the techique---")
    if raw=="YES" or raw=="yes" or raw=="y" or raw=="Yes":
        x1=input("Which Transfromation Technique you want to use, Stan/Norm::::")
        if x1=="Stan" or x1=="stan" or x1=="STAN":
            for i in integer_columns:
                X[i]=scaler.fit_transform(X[[i]])
        else:
            for i in integer_columns:
                X[i]=norm.fit_transform(X[[i]])
    else:
        pass
    #Label Encoder and Dummy Variable
    xx=input("Do you want to proceed with Label Encoder? Y or N::::")
    if xx=="Y" or xx=="y" or xx=="YES" or xx=="yes" or xx=="Yes":
        for i in object_columns:
            X[i]=le.fit_transform(X[i])
    else:
        for ii in object_columns:
            x1=pd.get_dummies(X[ii],drop_first=True)
            X=pd.concat([X,x1],axis=1)
        X.drop(columns=object_columns,inplace=True)
    knn_num=int(input("For how many times you want to iterate KNN Algorithm? "))
    knn=KNeighborsClassifier(n_neighbors=knn_num)
    print("The Accuracy of Logistic Regression with Monte Carlo simulation is--- ",np.mean(cross_val_score(log,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))
    print("The Accuracy of Random Forest Classification with Monte Carlo simulation is--- ",np.mean(cross_val_score(rfc,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))
    print("The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is--- ",np.mean(cross_val_score(knn,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))

# Bank Churner Data

In [4]:
bank=pd.read_csv("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\BankCustomers.csv").iloc[:,3:]
bank

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
bank.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
X=bank.drop(columns="Exited")
y=bank["Exited"]

In [10]:
X
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

#### Executing Monte-Carlo simulation

In [11]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---NO
Do you want to proceed with Label Encoder? Y or N::::Y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! YES

Accuracy of Logistic Regression is  0.796
Accuracy of RandomForestClassification is  0.865
Accuracy of NaiveBayesClassification is  0.79
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.7953333333333333 


In [12]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---YES
Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::Y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! YES

Accuracy of Logistic Regression is  0.816
Accuracy of RandomForestClassification is  0.863
Accuracy of NaiveBayesClassification is  0.834
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.8456666666666667 


In [13]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---YES
Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.814
Accuracy of RandomForestClassification is  0.866
Accuracy of NaiveBayesClassification is  0.834
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.8206666666666667 


In [15]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::n
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.796
Accuracy of RandomForestClassification is  0.866
Accuracy of NaiveBayesClassification is  0.789
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.7953333333333333 


In [17]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---No
Do you want to proceed with Label Encoder? Y or N::::Y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! No
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ROS

Accuracy of Logistic Regression is  0.634
Accuracy of RandomForestClassification is  0.861
Accuracy of NaiveBayesClassification is  0.702
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.69 


In [12]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::n
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS/RUS will gets activated!! yes

Accuracy of Logistic Regression is  0.814
Accuracy of RandomForestClassification is  0.865
Accuracy of NaiveBayesClassification is  0.826
For how many time you want to iterate KNN Algorithm? 20
Highest accuracy from K-Nearest Neighbor is 0.8206666666666667 


In [14]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS/RUS will gets activated!! ros
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros


NameError: name 'X_train' is not defined

In [120]:
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- rus

Accuracy of Logistic Regression on Test Data is  0.698
Accuracy of Logistic Regression on Training Data is  0.571 

Accuracy of RandomForestClassification on Test Data is  0.782
Accuracy of RandomForestClassification on Training Data is  0.691 

Accuracy of NaiveBayesClassification on Test Data is  0.738
Accuracy of NaiveBayesClassification on Test Data is  0.594


### Finalizing model with best transformation techniques combined with RandomOverSampler

In [141]:
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros

Accuracy of Logistic Regression on Test Data is  0.706
Accuracy of Logistic Regression on Training Data is  0.582 

Accuracy of RandomForestClassification on Test Data is  0.863
Accuracy of RandomForestClassification on Training Data is  0.707 

Accuracy of NaiveBayesClassification on Test Data is  0.728
Accuracy of NaiveBayesClassification on Test Data is  0.642


In [10]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Cross_Val(X,y)

How many batches of data you want in Monte Carlo simulation?  25
Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::stan
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 21
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.8077
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.8613
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.8419


In [12]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Cross_Val(X,y)

How many batches of data you want in Monte Carlo simulation?  25
Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 21
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.7902
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.8622
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.794


In [13]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Cross_Val(X,y)

How many batches of data you want in Monte Carlo simulation?  25
Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::n
For how many times you want to iterate KNN Algorithm? 21
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.8105
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.8626
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.8144000000000001


In [14]:
X=bank.drop(columns="Exited")
y=bank["Exited"]
ML_Cross_Val(X,y)

How many batches of data you want in Monte Carlo simulation?  21
Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 21
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.8078984214230767
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.8596014925886496
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.8138018310049387


#### From the above execution I am selecting the Random Forest Model with the transformation techniques as Normalization and Label Encoder because with Cross Validation and Monte-Carlo Simulations Random Forest is getting highest accuracy

In [18]:
bank=pd.read_csv("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\BankCustomers.csv").iloc[:,3:]
bank.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
X=bank.drop(columns=["Exited"])
y=bank["Exited"]

In [22]:
for i in ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']:
    X[i]=norm.fit_transform(X[[i]])

In [25]:
X["Geography"]=le.fit_transform(X["Geography"])
X["Gender"]=le.fit_transform(X["Gender"])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)
X_train.head()
y_train.head()

9136    0
6410    0
2607    0
3201    0
3330    0
Name: Exited, dtype: int64

In [30]:
#rfc=Random Forest Classifier
bank_model=rfc.fit(X_train,y_train)
bank_pred=bank_model.predict(X_test)

In [31]:
accuracy_score(y_test,bank_pred)

0.862

#### As you can see from the above model, we are getting accuracy at 86.20%
#### Now time to select best 5 features

In [34]:
x1=pd.DataFrame(bank_model.feature_importances_,columns=["Scores"])
x2=pd.DataFrame(X.columns,columns=["Names"])
pd.concat([x2,x1],axis=1).sort_values("Scores",ascending=False).head()

,Names,Scores
3,Age,0.234950
9,EstimatedSalary,0.149509
0,CreditScore,0.146024
5,Balance,0.142058
6,NumOfProducts,0.128080


#### After Selecting the Top 5 features, we are going to re-executing the model before the deployment, but this time the model gets fitted on the whole data instead of splitted data.

In [36]:
X=X[['Age', 'EstimatedSalary', 'CreditScore', 'Balance','NumOfProducts']]
X.head()

,Age,EstimatedSalary,CreditScore,Balance,NumOfProducts
0,0.324324,0.506735,0.538,0.000000,1
1,0.310811,0.562709,0.516,0.334031,1
2,0.324324,0.569654,0.304,0.636357,3
3,0.283784,0.469120,0.698,0.000000,2
4,0.337838,0.395400,1.000,0.500246,1


In [37]:
bank_model=rfc.fit(X,y)
bank_model.predict(X.tail(3))

array([1, 1, 0], dtype=int64)

In [38]:
y.tail(3)

9997    1
9998    1
9999    0
Name: Exited, dtype: int64

## As you can observe all 3 predictions comes accurate, hence I can deploy the model on the website, pickling the model in the local system

In [39]:
import pickle
pickle.dump(bank_model,open("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\BankCustomerML.pkl","wb"))

# Task Completes for Bank Customers Data
# Thanks

# Brain Data

## Data Importing

In [6]:
brain=pd.read_csv("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\brain_stroke.csv")
brain

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
4976,Male,41.0,0,0,No,Private,Rural,70.15,29.8,formerly smoked,0
4977,Male,40.0,0,0,Yes,Private,Urban,191.15,31.1,smokes,0
4978,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
4979,Male,40.0,0,0,Yes,Private,Rural,83.94,30.0,smokes,0


In [7]:
brain.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [8]:
X=brain.drop(columns=["stroke"])
y=brain['stroke']
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::stan
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 4733 that means the concentration of 0 category on overall data is 95.021%
The iteration of 1 category is 248 that means the concentration of 1 category on overall data is 5.0%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.951
Accuracy of RandomForestClassification is  0.95
Accuracy of NaiveBayesClassification is  0.864
For how many time you want to iterate KNN Algorithm? 25
Highest accuracy from K-Nearest Neighbor is 0.9505016722408027 


In [9]:
X=brain.drop(columns=["stroke"])
y=brain['stroke']
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---yes
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 4733 that means the concentration of 0 category on overall data is 95.021%
The iteration of 1 category is 248 that means the concentration of 1 category on overall data is 5.0%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.951
Accuracy of RandomForestClassification is  0.948
Accuracy of NaiveBayesClassification is  0.864
For how many time you want to iterate KNN Algorithm? 25
Highest accuracy from K-Nearest Neighbor is 0.9505016722408027 


In [10]:
X=brain.drop(columns=["stroke"])
y=brain['stroke']
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0 category is 4733 that means the concentration of 0 category on overall data is 95.021%
The iteration of 1 category is 248 that means the concentration of 1 category on overall data is 5.0%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.95
Accuracy of RandomForestClassification is  0.949
Accuracy of NaiveBayesClassification is  0.864
For how many time you want to iterate KNN Algorithm? 25
Highest accuracy from K-Nearest Neighbor is 0.9505016722408027 


In [11]:
X=brain.drop(columns=["stroke"])
y=brain['stroke']
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::n
The iteration of 0 category is 4733 that means the concentration of 0 category on overall data is 95.021%
The iteration of 1 category is 248 that means the concentration of 1 category on overall data is 5.0%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS OR ROS will gets activated!! yes

Accuracy of Logistic Regression is  0.951
Accuracy of RandomForestClassification is  0.949
Accuracy of NaiveBayesClassification is  0.835
For how many time you want to iterate KNN Algorithm? 25
Highest accuracy from K-Nearest Neighbor is 0.9505016722408027 


### Executing Cross Validation

In [126]:
X=brain.drop(columns="stroke")
y=brain["stroke"]
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.9506128514056223
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.949609638554217
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.9502112449799196


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,67.0,0,1,1,1,1,228.69,36.6,1
1,1,80.0,0,1,1,1,0,105.92,32.5,2
2,0,49.0,0,0,1,1,1,171.23,34.4,3
3,0,79.0,1,0,1,2,0,174.12,24.0,2
4,1,81.0,0,0,1,1,1,186.21,29.0,1
...,...,...,...,...,...,...,...,...,...,...
4976,1,41.0,0,0,0,1,0,70.15,29.8,1
4977,1,40.0,0,0,1,1,1,191.15,31.1,3
4978,0,45.0,1,0,1,0,0,95.02,31.8,3
4979,1,40.0,0,0,1,1,0,83.94,30.0,3


In [214]:
X=brain.drop(columns="stroke")
y=brain["stroke"]
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  10
Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::n
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.9504116667069077
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9484040369896419


In [216]:
X=brain.drop(columns="stroke")
y=brain["stroke"]
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  10
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::y
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.9502108634940564
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9457927904000772


In [217]:
X=brain.drop(columns="stroke")
y=brain["stroke"]
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  10
Which Transfromation Technique you want to use, Stan/Norm::::norm
Do you want to proceed with Label Encoder? Y or N::::n
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.9502108634940564
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9484040369896418


#### From the observations of above Executions I have come to the conclusion that since there is very very less difference in the Monte-Carlo simulation and Cross-Val simulation. I will go with Logistic Regression with Standardization and  with Label Encoder techique

#### Now Executing Logistic Regression Classifier on complete data

In [12]:
brain=pd.read_csv("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\brain_stroke.csv")
brain

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
4976,Male,41.0,0,0,No,Private,Rural,70.15,29.8,formerly smoked,0
4977,Male,40.0,0,0,Yes,Private,Urban,191.15,31.1,smokes,0
4978,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
4979,Male,40.0,0,0,Yes,Private,Rural,83.94,30.0,smokes,0


In [15]:
X=brain.drop(columns="stroke")
y=brain["stroke"]

In [16]:
for i in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    X[i]=le.fit_transform(X[i])

In [17]:
for i in ['avg_glucose_level', 'bmi', 'age']:
    X[i]=scaler.fit_transform(X[[i]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [19]:
X_train

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
4670,0,-0.415695,0,0,1,2,0,-0.383032,-0.677220,2
4605,1,-0.150917,1,0,1,2,1,-0.282745,-0.544667,3
4093,1,-1.210030,0,0,0,1,1,-0.563414,-0.736132,3
4871,1,-0.415695,0,0,1,1,0,-0.431178,0.206461,1
4592,0,-1.901984,0,0,0,3,1,0.059605,-1.310524,0
...,...,...,...,...,...,...,...,...,...,...
4049,0,-0.636344,0,0,0,1,1,-0.994292,-0.367931,2
1053,1,0.555158,0,0,1,1,1,-0.948808,0.442109,0
3526,0,0.422769,0,0,1,1,0,-1.002279,0.029725,3
3051,0,0.025601,0,0,1,1,0,-0.439166,-1.060148,2


In [20]:
model1=log.fit(X_train,y_train)

In [21]:
accuracy_score(y_test,model1.predict(X_test))

0.9505016722408027

In [23]:
x1=pd.DataFrame(model1.coef_).transpose().rename(columns={0:"Scores"})
x2=pd.DataFrame(X_train.columns,columns=["Names"])
pd.concat([x2,x1],axis=1).sort_values("Scores",ascending=False).head()

,Names,Scores
1,age,1.508813
2,hypertension,0.490383
3,heart_disease,0.247395
7,avg_glucose_level,0.211990
8,bmi,0.085003


In [25]:
X=X[['age', 'hypertension', 'heart_disease', 'avg_glucose_level','Residence_type']]

In [27]:
brain_model=log.fit(X,y)

In [28]:
pickle.dump(brain_model,open("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\Brain_stroke_ML.pkl","wb"))

In [541]:
y

0       1
1       1
2       1
3       1
4       1
       ..
4976    0
4977    0
4978    0
4979    0
4980    0
Name: stroke, Length: 4981, dtype: int64

# From the above execution, final model has been dumped to deploy on the website.

# Thanks

# HR Data

In [83]:
hr=pd.read_excel('C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\HR_data.xlsx')
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Sales_Occured,salary
0,0.38,0.53,2.0,157.0,3.0,0.0,1.0,0.0,sales,low
1,0.80,0.86,5.0,262.0,6.0,0.0,1.0,0.0,sales,medium
2,0.11,0.88,7.0,272.0,4.0,0.0,1.0,0.0,sales,medium
3,0.72,0.87,5.0,223.0,5.0,0.0,1.0,0.0,sales,low
4,0.37,0.52,2.0,159.0,3.0,0.0,1.0,0.0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2.0,151.0,3.0,0.0,1.0,0.0,support,low
14995,0.37,0.48,2.0,160.0,3.0,0.0,1.0,0.0,support,low
14996,0.37,0.53,2.0,143.0,3.0,0.0,1.0,0.0,support,low
14997,0.11,0.96,6.0,280.0,4.0,0.0,1.0,0.0,support,low


In [84]:
hr.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
Sales_Occured            0
salary                   0
dtype: int64

In [96]:
X=hr.drop(columns='left')
X
y=hr['left']
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
14994    1.0
14995    1.0
14996    1.0
14997    1.0
14998    1.0
Name: left, Length: 14999, dtype: float64

# Monte-Carlo Simulation

In [100]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0.0 category is 11428 that means the concentration of 0.0 category on overall data is 76.192%
The iteration of 1.0 category is 3571 that means the concentration of 1.0 category on overall data is 23.799999999999997%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros

Accuracy of Logistic Regression is  0.762
Accuracy of RandomForestClassification is  0.986
Accuracy of NaiveBayesClassification is  0.56
For how many time you want to iterate KNN Algorithm? 20
Hishest accuracy from K-Nearest Neighbor is 0.9622222222222222 


In [102]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::stan
Do you want to proceed with Label Encoder? Y or N::::n
The iteration of 0.0 category is 11428 that means the concentration of 0.0 category on overall data is 76.192%
The iteration of 1.0 category is 3571 that means the concentration of 1.0 category on overall data is 23.799999999999997%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros

Accuracy of Logistic Regression is  0.758
Accuracy of RandomForestClassification is  0.987
Accuracy of NaiveBayesClassification is  0.577
For how many time you want to iterate KNN Algorithm? 20
Hishest accuracy from K-Nearest Neighbor is 0.96 


In [103]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::y
The iteration of 0.0 category is 11428 that means the concentration of 0.0 category on overall data is 76.192%
The iteration of 1.0 category is 3571 that means the concentration of 1.0 category on overall data is 23.799999999999997%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros

Accuracy of Logistic Regression is  0.759
Accuracy of RandomForestClassification is  0.986
Accuracy of NaiveBayesClassification is  0.572
For how many time you want to iterate KNN Algorithm? 20
Hishest accuracy from K-Nearest Neighbor is 0.96 


In [104]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Monte_Carlo(X,y)

Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::n
The iteration of 0.0 category is 11428 that means the concentration of 0.0 category on overall data is 76.192%
The iteration of 1.0 category is 3571 that means the concentration of 1.0 category on overall data is 23.799999999999997%
Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ros

Accuracy of Logistic Regression is  0.761
Accuracy of RandomForestClassification is  0.987
Accuracy of NaiveBayesClassification is  0.58
For how many time you want to iterate KNN Algorithm? 20
Hishest accuracy from K-Nearest Neighbor is 0.96 


# Cross-Validation Simulation

In [105]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.7627128615932355
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9927999999999999
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.962931464174455


In [106]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Which Transfromation Technique you want to use, Stan/Norm::::Stan
Do you want to proceed with Label Encoder? Y or N::::n
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.78798202047174
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9926
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.9449970627503337


In [107]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.7631796172674676
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9927999999999999
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.9619312861593234


In [109]:
X=hr.drop(columns='left')
X
y=hr['left']
y
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Which Transfromation Technique you want to use, Stan/Norm::::Norm
Do you want to proceed with Label Encoder? Y or N::::n
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.7873153538050734
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.9923333333333334
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.9297956386292835


#### From the Monte-carlo and Cross validation simulation I have picked Random Forest Algorithm and would like to deploy the same algorithm on the website

In [338]:
X=hr.drop(columns='left')
X
y=hr['left']

In [339]:
for i in X[["Sales_Occured","salary"]]:
    X[i]=le.fit_transform(X[[i]])

In [341]:
X.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Sales_Occured,salary
0,0.38,0.53,2.0,157.0,3.0,0.0,0.0,7,1
1,0.80,0.86,5.0,262.0,6.0,0.0,0.0,7,2
2,0.11,0.88,7.0,272.0,4.0,0.0,0.0,7,2
3,0.72,0.87,5.0,223.0,5.0,0.0,0.0,7,1
4,0.37,0.52,2.0,159.0,3.0,0.0,0.0,7,1


In [342]:
rfc.fit(X,y)

RandomForestClassifier()

In [352]:
x1=pd.DataFrame(rfc.feature_importances_,columns=["Scores"])
x2=pd.DataFrame(X.columns,columns=["Names"])
pd.concat([x2,x1],axis=1).sort_values("Scores",ascending=False).head(6)["Names"].values

array(['satisfaction_level', 'time_spend_company', 'number_project',
       'average_montly_hours', 'last_evaluation', 'Sales_Occured'],
      dtype=object)

In [355]:
X=X[['satisfaction_level', 'time_spend_company', 'number_project',
       'average_montly_hours', 'last_evaluation', 'Sales_Occured']]
X

,satisfaction_level,time_spend_company,number_project,average_montly_hours,last_evaluation,Sales_Occured
0,0.38,3.0,2.0,157.0,0.53,7
1,0.80,6.0,5.0,262.0,0.86,7
2,0.11,4.0,7.0,272.0,0.88,7
3,0.72,5.0,5.0,223.0,0.87,7
4,0.37,3.0,2.0,159.0,0.52,7
...,...,...,...,...,...,...
14994,0.40,3.0,2.0,151.0,0.57,8
14995,0.37,3.0,2.0,160.0,0.48,8
14996,0.37,3.0,2.0,143.0,0.53,8
14997,0.11,4.0,6.0,280.0,0.96,8


In [357]:
hr_model=rfc.fit(X,y)
hr_model
pickle.dump(hr_model,open("C:\\Users\\Karan\\Desktop\\DBS Casual Assessments\\hr_model.pkl","wb"))

# From the above execution we have dumped the model and is ready to be deployed on the website

# Thank You

In [2]:
def ML_Cross_Val(X,y):
    object_columns=X.dtypes[X.dtypes==np.object].index
    integer_columns=X.dtypes[(X.dtypes==np.int64) | (X.dtypes==np.float64)].index    
    
    #Standardization and Normalization
    repeat=int(input("How much repeatition you want in Monte Carlo simulation?  "))
    raw=input("Do you want to transform the data, if YES then please specify the techique---")
    if raw=="YES" or raw=="yes" or raw=="y" or raw=="Yes":
        x1=input("Which Transfromation Technique you want to use, Stan/Norm::::")
        if x1=="Stan" or x1=="stan" or x1=="STAN":
            for i in integer_columns:
                X[i]=scaler.fit_transform(X[[i]])
        else:
            for i in integer_columns:
                X[i]=norm.fit_transform(X[[i]])
    else:
        pass
    #Label Encoder and Dummy Variable
    xx=input("Do you want to proceed with Label Encoder? Y or N::::")
    if xx=="Y" or xx=="y" or xx=="YES" or xx=="yes" or xx=="Yes":
        for i in object_columns:
            X[i]=le.fit_transform(X[i])
    else:
        for ii in object_columns:
            x1=pd.get_dummies(X[ii],drop_first=True)
            X=pd.concat([X,x1],axis=1)
        X.drop(columns=object_columns,inplace=True)
    knn_num=int(input("For how many times you want to iterate KNN Algorithm? "))
    knn=KNeighborsClassifier(n_neighbors=knn_num)
    print("The Accuracy of Logistic Regression with Monte Carlo simulation is--- ",np.mean(cross_val_score(log,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))
    print("The Accuracy of Random Forest Classification with Monte Carlo simulation is--- ",np.mean(cross_val_score(rfc,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))
    print("The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is--- ",np.mean(cross_val_score(knn,X,y,cv=repeat,scoring='accuracy',n_jobs=-1)))
    return X

In [3]:
bank=pd.read_csv("C:\\Users\\Karan\\Desktop\\Data\\BankCustomers.csv").iloc[:,3:]
bank

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
X=bank.drop(columns='Exited')
y=bank["Exited"]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [6]:
X=bank.drop(columns='Exited')
y=bank["Exited"]
ML_Cross_Val(X,y)

How much repeatition you want in Monte Carlo simulation?  20
Do you want to transform the data, if YES then please specify the techique---no
Do you want to proceed with Label Encoder? Y or N::::y
For how many times you want to iterate KNN Algorithm? 20
The Accuracy of Logistic Regression with Monte Carlo simulation is---  0.7901
The Accuracy of Random Forest Classification with Monte Carlo simulation is---  0.8624
The Accuracy of K-Nearest Neighbor Classifier with Monte Carlo simualtion is---  0.7956000000000001


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64
9996,516,0,1,35,10,57369.61,1,1,1,101699.77
9997,709,0,0,36,7,0.00,1,0,1,42085.58
9998,772,1,1,42,3,75075.31,2,1,0,92888.52


In [18]:
def ML_Monte_Carlo(X,y):
    object_columns=X.dtypes[X.dtypes==np.object].index
    integer_columns=X.dtypes[(X.dtypes==np.int64) | (X.dtypes==np.float64)].index
    #Standardization, Normalization and RAW Data
    raw=input("Do you want to transform the data, if YES then please specify the techique---")
    if raw=="YES" or raw=="yes" or raw=="y" or raw=="Yes" or raw=="Y":
        x1=input("Which Transfromation Technique you want to use, Stan/Norm::::")
        if x1=="Stan" or x1=="stan" or x1=="STAN":
            for i in integer_columns:
                X[i]=scaler.fit_transform(X[[i]])
        else:
            for i in integer_columns:
                X[i]=norm.fit_transform(X[[i]])
    else:
        pass
    #Label Encoder and Dummy Variable
    xx=input("Do you want to proceed with Label Encoder? Y or N::::")
    if xx=="Y" or xx=="y" or xx=="YES" or xx=="yes" or xx=="Yes":
        for i in object_columns:
            X[i]=le.fit_transform(X[i])
    else:
        for ii in object_columns:
            x1=pd.get_dummies(X[ii],drop_first=True)
            X=pd.concat([X,x1],axis=1)
        X.drop(columns=object_columns,inplace=True)
    #Treating the Over and Under Samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    first=y.value_counts().index[0]
    first_iter=y.value_counts().values[0]
    second=y.value_counts().index[1]
    second_iter=y.value_counts().values[1]
    print('The iteration of {} category is {} that means the concentration of {} category on overall data is {}%'.format(first,first_iter,first,np.round((first_iter/y.shape[0])*100,3)))
    print("The iteration of {} category is {} that means the concentration of {} category on overall data is {}%".format(second,second_iter,second,np.round(second_iter/y.shape[0],3)*100))
    if np.round(first_iter/y.shape[0],3)>=0.65:
        imbalance=input("Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS/RUS will gets activated!! ")
        if imbalance=="Yes" or imbalance=="YES" or imbalance=="Y" or imbalance=="y" or imbalance=="yes":
            X_train_reshape, X_test, y_train_reshape, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)
        else:
            answer=input("Do you want to continue with Random Over Sampling or Random Under Sampling please answer in ROS/ros or RUS/rus---- ")
            if answer=="ROS" or answer=="ros" or answer=="Ros":
                X_train_reshape,y_train_reshape=ros.fit_resample(X_train,y_train)
            else:
                X_train_reshape,y_train_reshape=rus.fit_resample(X_train,y_train)
    else:
        pass
    return X_train_reshape

In [20]:
X=bank.drop(columns='Exited')
y=bank["Exited"]
ML_Monte_Carlo(X,y)

Do you want to transform the data, if YES then please specify the techique---NO
Do you want to proceed with Label Encoder? Y or N::::N
The iteration of 0 category is 7963 that means the concentration of 0 category on overall data is 79.63%
The iteration of 1 category is 2037 that means the concentration of 1 category on overall data is 20.4%
Imbalance Target Output has been detected, for Stratified Random Sampling type YES else RUS/RUS will gets activated!! YES


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
9136,595,20,4,95830.43,1,1,0,177738.98,0,0,1
6410,689,63,1,0.00,2,1,1,186526.12,0,1,0
2607,814,31,1,118870.92,1,1,0,101704.19,0,0,1
3201,722,20,6,0.00,2,1,0,195486.28,0,0,1
3330,738,18,4,0.00,2,1,1,47799.15,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1467,624,35,0,180303.24,2,1,0,163587.90,1,0,0
4644,712,74,5,0.00,2,0,0,151425.82,0,1,1
8942,667,37,9,71786.90,2,1,1,67734.79,0,0,0
2935,687,35,8,100988.39,2,1,0,22247.27,0,1,1
